# Trading Hours, Spreads and Granularity - control and limit Trading Costs

__Goal__: Finding the right Trading hours and granularity to control and limit costs.

__Problem__: We can´t forecast returns with high accuracy -> In all cases where we predict market direction correctly, price movements/volatility must be large enough to cover trading costs.

__Solution__: <br> 
-Make Trades only during busy Trading hours (increased Volatility)<br>
-Lower Granularity leads to larger price movements per bar/candle (to cover Trading Costs) 


__Trade Off__: The higher the Granularity the more likely it is to find markets inefficiencies and mispricing.

## Getting and Preparing the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [3]:
df = pd.read_csv("bid_ask.csv", parse_dates = ["time"], index_col = "time")

In [4]:
df

,volume,ask,bid,spread,mid
time,,,,,
2019-07-21 21:00:00+00:00,12,1.12164,1.12131,0.00033,1.121475
2019-07-21 21:05:00+00:00,12,1.12182,1.12157,0.00025,1.121695
2019-07-21 21:10:00+00:00,1,1.12176,1.12158,0.00018,1.121670
2019-07-21 21:15:00+00:00,18,1.12190,1.12141,0.00049,1.121655
2019-07-21 21:20:00+00:00,9,1.12201,1.12167,0.00034,1.121840
...,...,...,...,...,...
2020-10-20 23:35:00+00:00,81,1.18275,1.18261,0.00014,1.182680
2020-10-20 23:40:00+00:00,92,1.18276,1.18263,0.00013,1.182695
2020-10-20 23:45:00+00:00,101,1.18275,1.18262,0.00013,1.182685


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 93021 entries, 2019-07-21 21:00:00+00:00 to 2020-10-20 23:55:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   volume  93021 non-null  int64  
 1   ask     93021 non-null  float64
 2   bid     93021 non-null  float64
 3   spread  93021 non-null  float64
 4   mid     93021 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 4.3 MB


In [6]:
df.index.tz

datetime.timezone.utc

In [7]:
df["NYTime"] = df.index.tz_convert("America/New_York")

In [8]:
df

,volume,ask,bid,spread,mid,NYTime
time,,,,,,
2019-07-21 21:00:00+00:00,12,1.12164,1.12131,0.00033,1.121475,2019-07-21 17:00:00-04:00
2019-07-21 21:05:00+00:00,12,1.12182,1.12157,0.00025,1.121695,2019-07-21 17:05:00-04:00
2019-07-21 21:10:00+00:00,1,1.12176,1.12158,0.00018,1.121670,2019-07-21 17:10:00-04:00
2019-07-21 21:15:00+00:00,18,1.12190,1.12141,0.00049,1.121655,2019-07-21 17:15:00-04:00
2019-07-21 21:20:00+00:00,9,1.12201,1.12167,0.00034,1.121840,2019-07-21 17:20:00-04:00
...,...,...,...,...,...,...
2020-10-20 23:35:00+00:00,81,1.18275,1.18261,0.00014,1.182680,2020-10-20 19:35:00-04:00
2020-10-20 23:40:00+00:00,92,1.18276,1.18263,0.00013,1.182695,2020-10-20 19:40:00-04:00
2020-10-20 23:45:00+00:00,101,1.18275,1.18262,0.00013,1.182685,2020-10-20 19:45:00-04:00


In [9]:
df["hour"] = df.NYTime.dt.hour

In [10]:
df

,volume,ask,bid,spread,mid,NYTime,hour
time,,,,,,,
2019-07-21 21:00:00+00:00,12,1.12164,1.12131,0.00033,1.121475,2019-07-21 17:00:00-04:00,17
2019-07-21 21:05:00+00:00,12,1.12182,1.12157,0.00025,1.121695,2019-07-21 17:05:00-04:00,17
2019-07-21 21:10:00+00:00,1,1.12176,1.12158,0.00018,1.121670,2019-07-21 17:10:00-04:00,17
2019-07-21 21:15:00+00:00,18,1.12190,1.12141,0.00049,1.121655,2019-07-21 17:15:00-04:00,17
2019-07-21 21:20:00+00:00,9,1.12201,1.12167,0.00034,1.121840,2019-07-21 17:20:00-04:00,17
...,...,...,...,...,...,...,...
2020-10-20 23:35:00+00:00,81,1.18275,1.18261,0.00014,1.182680,2020-10-20 19:35:00-04:00,19
2020-10-20 23:40:00+00:00,92,1.18276,1.18263,0.00013,1.182695,2020-10-20 19:40:00-04:00,19
2020-10-20 23:45:00+00:00,101,1.18275,1.18262,0.00013,1.182685,2020-10-20 19:45:00-04:00,19


In [11]:
df["price_change_abs"] = df.mid.diff().abs()

In [12]:
df

,volume,ask,bid,spread,mid,NYTime,hour,price_change_abs
time,,,,,,,,
2019-07-21 21:00:00+00:00,12,1.12164,1.12131,0.00033,1.121475,2019-07-21 17:00:00-04:00,17,NaN
2019-07-21 21:05:00+00:00,12,1.12182,1.12157,0.00025,1.121695,2019-07-21 17:05:00-04:00,17,0.000220
2019-07-21 21:10:00+00:00,1,1.12176,1.12158,0.00018,1.121670,2019-07-21 17:10:00-04:00,17,0.000025
2019-07-21 21:15:00+00:00,18,1.12190,1.12141,0.00049,1.121655,2019-07-21 17:15:00-04:00,17,0.000015
2019-07-21 21:20:00+00:00,9,1.12201,1.12167,0.00034,1.121840,2019-07-21 17:20:00-04:00,17,0.000185
...,...,...,...,...,...,...,...,...
2020-10-20 23:35:00+00:00,81,1.18275,1.18261,0.00014,1.182680,2020-10-20 19:35:00-04:00,19,0.000100
2020-10-20 23:40:00+00:00,92,1.18276,1.18263,0.00013,1.182695,2020-10-20 19:40:00-04:00,19,0.000015
2020-10-20 23:45:00+00:00,101,1.18275,1.18262,0.00013,1.182685,2020-10-20 19:45:00-04:00,19,0.000010


In [13]:
df.dropna(inplace = True)

## The best time to trade (Part 1)

In [ ]:
by_hour = df.groupby("hour")[["volume", "spread", "price_change_abs"]].mean()
by_hour

In [ ]:
by_hour.volume.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Trading Volumne EUR/USD", fontsize = 15)
plt.title("Trading Volume", fontsize = 15)
plt.show()

In [ ]:
by_hour.spread.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Spread EUR/USD", fontsize = 15)
plt.title("Spread", fontsize = 15)
plt.show()

In [ ]:
by_hour.loc[0:16, "spread"].plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Spread EUR/USD", fontsize = 15)
plt.title("Spread", fontsize = 15)
plt.show()

In [ ]:
by_hour.price_change_abs.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Price Changes EUR/USD", fontsize = 15)
plt.title("Price Changes", fontsize = 15)
plt.show()

## The best time to trade (Part 2)

In [ ]:
df

In [ ]:
df["cover_cost"] = df.price_change_abs > df.spread

In [ ]:
df

In [ ]:
df.groupby("hour").cover_cost.mean()

In [ ]:
df.groupby("hour").cover_cost.mean().plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Percentage of Bars where Costs are covered", fontsize = 15)
plt.show()

Busy Trading Hours for __EUR/USD__: from __2:00 am__ to __12:59 pm__ New York Time

## Proportional Trading Costs and Trading Hours

In [ ]:
ptc = 0.00007

In [ ]:
ptc

In [ ]:
df

In [ ]:
df.spread.mean()

In [ ]:
spread = df.loc[df.hour.between(2, 12), "spread"].mean()
spread

In [ ]:
ptc = (spread/2) / df.mid.mean()
ptc

## The Impact of Granularity

In [ ]:
def hours_granularity(freq = None):
    df = pd.read_csv("bid_ask.csv", parse_dates = ["time"], index_col = "time",
                     usecols = ["time", "spread", "mid"])
    if freq is not None:
        df = df.resample(freq).last().dropna()
    df["NYTime"] = df.index.tz_convert("America/New_York")
    df["hour"] = df.NYTime.dt.hour
    df["price_change_abs"] = df.mid.diff().abs()
    df["cover_cost"] = df.price_change_abs > df.spread
    
    df.dropna().groupby("hour").cover_cost.mean().plot(kind = "bar", figsize = (12, 8), fontsize = 13)
    plt.xlabel("NY Time", fontsize = 15)
    plt.ylabel("Cover Costs", fontsize = 15)
    plt.title("Granularity: {}".format(freq), fontsize = 18)
    plt.yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
    plt.show()

In [ ]:
hours_granularity(freq = "5min")

In [ ]:
hours_granularity(freq = "10min")

In [ ]:
hours_granularity(freq = "20min")

In [ ]:
hours_granularity(freq = "30min")

In [ ]:
# hours_granularity(freq = "1H") # old

In [ ]:
hours_granularity(freq = "1h") # new (Pandas 3.x)